In [1]:
import pickle
from pathlib import Path
from importlib import reload

from bikewaysim.paths import config
from bikewaysim.impedance_calibration import stochastic_optimization, loss_functions
from bikewaysim.routing import rustworkx_routing_funcs

with (config['calibration_fp']/'subsets.pkl').open('rb') as fh:
    subsets = pickle.load(fh)
subsets = [x for x in subsets if x[0] == 'debug']

kwargs = {
    'calibration_name': 'debug',    
    'betas_tup': (
        {'col':'2lpd','type':'link','range':[0,3]},
        {'col':'3+lpd','type':'link','range':[0,3]},
        {'col':'(30,inf) mph','type':'link','range':[0,3]},
        {'col':'[4k,10k) aadt','type':'link','range':[0,3]},
        {'col':'[10k,inf) aadt','type':'link','range':[0,3]},
        {'col':'[4,6) grade','type':'link','range':[0,3]},
        {'col':'[6,inf) grade','type':'link','range':[0,3]},
        {'col':'bike lane','type':'link','range':[-1,3]},
        {'col':'multi use path and cycletrack','type':'link','range':[-1,3]},
        {'col':'unsig_crossing','type':'turn','range':[0,5]},
        {'col':'left_turn','type':'turn','range':[0,5]}
    ),
    'set_to_zero': ['bike lane','cycletrack','multi use path'],
    'set_to_inf': ['not_street'],
    'objective_function': loss_functions.jaccard_buffer_total,
    'stochastic_optimization_settings': {'method':'pso','options':{'maxiter':10,'popsize':3,'return_all':True,'ftol':5}},
    'print_results': True,
    'subset': subsets[0]
}

#BUG if last results resulted in negative edge weights, then the post caliiration will fail

In [5]:
import numpy as np
test = [np.inf,2]
np.argmin(test)

1

In [2]:
reload(stochastic_optimization)
stochastic_optimization.full_impedance_calibration(**kwargs)

['2lpd', '3+lpd', '(30,inf) mph', '[4k,10k) aadt', '[10k,inf) aadt', '[4,6) grade', '[6,inf) grade', 'bike lane', 'multi use path and cycletrack', 'unsig_crossing', 'left_turn', 'objective_function']
[0.31, 0.24, 0.04, 1.21, 0.21, 2.42, 0.18, 0.85, 0.64, 0.29, 3.66, -0.18]
[2.1, 1.43, 2.5, 0.25, 1.15, 1.06, 1.13, 2.06, -0.83, 2.48, 1.91, -0.2]
[1.01, 2.46, 1.43, 2.05, 2.24, 0.2, 2.21, -0.42, 2.13, 3.37, 0.19, -0.09]
[1.89, 1.78, 1.66, -0.07, 1.02, 0.71, 0.87, 1.16, 0.24, 1.68, 2.47, -0.17]
[2.1, 1.43, 2.5, 0.25, 1.15, 1.06, 1.13, 2.06, -0.83, 2.48, 1.91, -0.2]
[1.1, 2.43, 2.64, 0.51, 0.96, 1.01, 2.07, 1.93, 0.75, 3.24, 0.63, -0.11]
[2.94, 1.89, 1.68, -0.44, 1.42, 0.93, 1.3, 2.31, -0.47, 3.35, 2.56, -0.19]
[2.1, 1.43, 2.5, 0.25, 1.15, 1.06, 1.13, 2.06, -0.83, 2.48, 1.91, -0.2]
[1.58, 1.89, 3.32, -0.71, 0.16, 1.6, 1.6, 3.71, -0.38, 2.45, 2.06, -0.19]
Duration: 0 days 00:03:40
jaccard_buffer_total: -0.2013
     fun: -0.2013
 message: 'maximum number of iterations is reached'
    nfev: 9
 

False

In [6]:
with (config['calibration_fp']/'results/debug,debug,7.pkl').open('rb') as fh:
    test = pickle.load(fh)

In [16]:
funall = test['results'].funall
# np.argmax(funall,axis=1)
funall

array([[5., 3., 3.],
       [8., 8., 0.]])

In [8]:
test['results'].xall

array([[[ 2.06223899e-01,  1.07184735e+00,  2.12601294e+00,
          1.15557930e+00,  2.39061116e+00,  1.21728709e+00,
          3.73823614e-02,  2.04248770e+00,  5.31220771e-01,
          1.98525450e+00,  2.63591503e-01],
        [ 2.16990392e+00,  2.41646121e+00,  1.40368413e+00,
          2.77924496e-01,  3.67763976e-02,  2.09504654e+00,
          2.21848374e+00,  6.74652860e-01, -3.39962966e-01,
          3.42372480e+00,  3.82646351e+00],
        [ 1.22080636e+00,  5.07564789e-02,  2.62283854e-01,
          2.08002960e+00,  1.09995561e+00,  3.70408507e-01,
          1.25381697e+00, -8.00182100e-01,  2.00915582e+00,
          3.44708487e-01,  2.28838730e+00]],

       [[ 2.80181491e-01,  2.12551504e+00,  2.03552385e+00,
          2.07086632e-01,  1.21738959e+00,  2.24839126e+00,
          2.75745322e+00,  2.11268095e-01,  2.47643818e-04,
          3.41463577e+00,  3.06768247e-01],
        [ 2.16990392e+00,  2.41646121e+00,  1.40368413e+00,
          2.77924496e-01,  3.67763976e-02,

In [ ]:
np.linalg.norm(xbest - x[idx]) <= xtol

In [ ]:
# figure out how to calculate xtol

In [35]:
import numpy as np

def rosenbrock(x):
   x = np.asarray(x)
   sum1 = ((x[1:] - x[:-1] ** 2) ** 2).sum()
   sum2 = np.square(1.0 - x[:-1]).sum()
   return 100.0 * sum1 + sum2

bounds = [[-5.12, 5.12], [-5.12, 5.12]]  # The number of variables to optimize is len(bounds)

from stochopy.optimize import minimize

x = minimize(rosenbrock, bounds, method="cmaes", options={"maxiter": 100, "popsize": 10, "seed": 0, 'return_all': True, "xtol": 2,"ftol": 0})

In [ ]:
x

In [ ]:
x.xall[0]

In [5]:
with (config['calibration_fp']/'results/all,gdot,1.pkl').open('rb') as fh:
    test = pickle.load(fh)

In [6]:
test['results']

     fun: -0.4169325701134658
 message: 'best solution value is lower than ftol'
    nfev: 2375
     nit: 95
  status: 1
 success: True
       x: array([-0.39195709, -0.18004657,  0.00310853,  0.2925294 ,  0.09739835])

In [12]:
import numpy as np
np.array(test['past_vals'])[:,-1]

array([-0.32460515, -0.27911411, -0.32388118, ..., -0.41630012,
       -0.41552668, -0.41693257])